In [20]:
###Logistic Regression & RForests

from collections import Counter
import pandas as pd

#Handle Imbalance Data
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import smote_variants as sv

#ML ALgorithims
from matplotlib import pyplot
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import RandomizedSearchCV

In [21]:
#Reading our data

train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

/Users/leotsang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
train.columns

Index(['player_name', 'school', 'conference', 'GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'rec-rk', 'ast/tov', 'rimmade',
       'rimmade + rimmiss', 'midmade', 'midmade + midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+mismiss)', 'dunksmade',
       'dunksmiss + dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'pos', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'ht_in',
       'drafted', 'yr_cat', 'ATH', 'GP_adj', 'BBIQ'],
      dtype='object')

In [43]:
#stats to look at

per_stats = ['Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FT_per', '2P_per', '3P_per', 
             'blk_per', 'stl_per', 'GP', 'ht_in', 'yr_cat',  'ATH', 'BBIQ']
box_stats = ['GP', 'mp', 'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg','oreb', 'dreb', 'treb',
             'ast', 'stl', 'blk', 'ht_in', 'yr_cat', 'FTA', 'FTM', 'ftr',  'ATH', 'BBIQ']
adv_stats = ['GP', 'mp', 'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'oreb', 'dreb', 'treb',
             'ast', 'stl', 'blk', 'ht_in', 'yr_cat', 'bpm', 'obpm', 'dbpm', 'FTA', 'FTM', 'ftr' ,'ATH', 'BBIQ']

per_adv = ['Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FT_per', '2P_per', '3P_per', 
             'blk_per', 'stl_per', 'GP', 'ht_in', 'yr_cat', 'bpm', 'obpm', 'dbpm', 'ATH', 'BBIQ']

all_stats = ['GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'porpag', 'adjoe',
       'pfr','year', 'rec-rk', 'ast/tov', 
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'yr_cat', 'ht_in']

In [24]:
#delete all missing values
train = train.dropna(axis=0, subset=per_stats)
train = train.dropna(axis=0, subset=box_stats)
train = train.dropna(axis=0, subset=adv_stats)

#delete mssing value in test set as well

test = test.dropna(axis=0, subset=per_stats)
test = test.dropna(axis=0, subset=box_stats)
test = test.dropna(axis=0, subset=adv_stats)

In [25]:
#set our 2 test-set aside
test_2018 = test[test['year']==2018]
test_2019 = test[test['year']==2019]

In [39]:
tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats

In [44]:
#our preliminary model: Log Reg

for i in tests:
    
    model = LogisticRegression(max_iter=10000)
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))


F1: 0.07142857142857142 recall: 0.0392156862745098 precision: 0.4 predictions: 5.0
F1: 0.30588235294117644 recall: 0.2549019607843137 precision: 0.38235294117647056 predictions: 34.0
F1: 0.24657534246575347 recall: 0.17647058823529413 precision: 0.4090909090909091 predictions: 22.0
F1: 0.3333333333333333 recall: 0.21568627450980393 precision: 0.7333333333333333 predictions: 15.0


In [45]:
#preliminary Random Forest

for i in tests:
    
    model = RandomForestClassifier()
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))


F1: 0.0 recall: 0.0 precision: 0.0 predictions: 6.0
F1: 0.09836065573770492 recall: 0.058823529411764705 precision: 0.3 predictions: 10.0
F1: 0.3384615384615385 recall: 0.21568627450980393 precision: 0.7857142857142857 predictions: 14.0
F1: 0.19047619047619047 recall: 0.11764705882352941 precision: 0.5 predictions: 12.0


In [46]:
#preliminary  Adaboost #EXP_ID 1

for i in tests:
    
    model = AdaBoostClassifier()
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))


F1: 0.24096385542168675 recall: 0.19607843137254902 precision: 0.3125 predictions: 32.0
F1: 0.32941176470588235 recall: 0.27450980392156865 precision: 0.4117647058823529 predictions: 34.0
F1: 0.46511627906976744 recall: 0.39215686274509803 precision: 0.5714285714285714 predictions: 35.0
F1: 0.273972602739726 recall: 0.19607843137254902 precision: 0.45454545454545453 predictions: 22.0


In [47]:
###preliminary MLP

for i in tests:
    
    model = MLPClassifier()
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

F1: 0.19753086419753085 recall: 0.1568627450980392 precision: 0.26666666666666666 predictions: 30.0
F1: 0.21538461538461542 recall: 0.13725490196078433 precision: 0.5 predictions: 14.0
F1: 0.2068965517241379 recall: 0.11764705882352941 precision: 0.8571428571428571 predictions: 7.0
F1: 0.3614457831325301 recall: 0.29411764705882354 precision: 0.46875 predictions: 32.0


In [48]:
#with Smote LogREG####

for i in tests:
    
    model = LogisticRegression(max_iter=10000)
    oversample = SMOTE(sampling_strategy=0.05)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))



F1: 0.38095238095238093 recall: 0.5490196078431373 precision: 0.2916666666666667 predictions: 96.0
F1: 0.3668639053254438 recall: 0.6078431372549019 precision: 0.2627118644067797 predictions: 118.0
F1: 0.4246575342465754 recall: 0.6078431372549019 precision: 0.3263157894736842 predictions: 95.0
F1: 0.37593984962406013 recall: 0.49019607843137253 precision: 0.3048780487804878 predictions: 82.0


In [49]:
###WITH SMOTE XGBOOOST##### EXP_ID 2

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = XGBClassifier(max_depth=4)
    oversample = SMOTE(sampling_strategy=0.05)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

F1: 0.3269230769230769 recall: 0.3333333333333333 precision: 0.32075471698113206 predictions: 53.0
F1: 0.3809523809523809 recall: 0.39215686274509803 precision: 0.37037037037037035 predictions: 54.0
F1: 0.48 recall: 0.47058823529411764 precision: 0.4897959183673469 predictions: 49.0
F1: 0.44680851063829785 recall: 0.4117647058823529 precision: 0.4883720930232558 predictions: 43.0


In [50]:
###WITH SMOTE MLP##### EXP_ID: 3

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = MLPClassifier()
    oversample = SMOTE(sampling_strategy=0.025)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

F1: 0.18018018018018017 recall: 0.19607843137254902 precision: 0.16666666666666666 predictions: 60.0
F1: 0.27129337539432175 recall: 0.8431372549019608 precision: 0.16165413533834586 predictions: 266.0
F1: 0.47619047619047616 recall: 0.39215686274509803 precision: 0.6060606060606061 predictions: 33.0
F1: 0.3225806451612903 recall: 0.39215686274509803 precision: 0.273972602739726 predictions: 73.0


In [56]:
###ADA Boost SMOTE(& NO SMOTE ARE THE SAME) EXP_ID: 4

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = AdaBoostClassifier()
    oversample = SMOTE(sampling_strategy=0.4)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

F1: 0.2608695652173913 recall: 0.23529411764705882 precision: 0.2926829268292683 predictions: 41.0
F1: 0.33333333333333337 recall: 0.29411764705882354 precision: 0.38461538461538464 predictions: 39.0
F1: 0.5057471264367817 recall: 0.43137254901960786 precision: 0.6111111111111112 predictions: 36.0
F1: 0.30769230769230765 recall: 0.23529411764705882 precision: 0.4444444444444444 predictions: 27.0


In [53]:
###WITH SMOTE XGBOOOST##### & UNDERSAMPLING EXP_ID 5

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = XGBClassifier(max_depth=4)
    over = SMOTE(sampling_strategy=0.05)
    under = RandomUnderSampler(sampling_strategy=0.05)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

F1: 0.27999999999999997 recall: 0.27450980392156865 precision: 0.2857142857142857 predictions: 49.0
F1: 0.41441441441441446 recall: 0.45098039215686275 precision: 0.38333333333333336 predictions: 60.0
F1: 0.46 recall: 0.45098039215686275 precision: 0.46938775510204084 predictions: 49.0
F1: 0.4444444444444445 recall: 0.43137254901960786 precision: 0.4583333333333333 predictions: 48.0


In [80]:
###MLP Under and Over Sample EXP_ID: 6

for i in tests:
    
    model = MLPClassifier()
    over = SMOTE(sampling_strategy=0.05)
    under = RandomUnderSampler(sampling_strategy=0.05)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

F1: 0.25510204081632654 recall: 0.49019607843137253 precision: 0.1724137931034483 predictions: 145.0
F1: 0.41624365482233505 recall: 0.803921568627451 precision: 0.2808219178082192 predictions: 146.0
F1: 0.42105263157894735 recall: 0.5490196078431373 precision: 0.34146341463414637 predictions: 82.0
F1: 0.3354838709677419 recall: 0.5098039215686274 precision: 0.25 predictions: 104.0


In [55]:
#with Smote LogREG#### & Undersample EXP_ID 7

for i in tests:
    
    model = LogisticRegression(max_iter=10000)
    over = SMOTE(sampling_strategy=0.05)
    under = RandomUnderSampler(sampling_strategy=0.05)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    #roc_auc = roc_auc_score(test_2019['drafted'], predictions)
    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))



F1: 0.3741935483870968 recall: 0.5686274509803921 precision: 0.27884615384615385 predictions: 104.0
F1: 0.38036809815950917 recall: 0.6078431372549019 precision: 0.2767857142857143 predictions: 112.0
F1: 0.4217687074829932 recall: 0.6078431372549019 precision: 0.3229166666666667 predictions: 96.0
F1: 0.373134328358209 recall: 0.49019607843137253 precision: 0.30120481927710846 predictions: 83.0


In [83]:
####SMOTE & UNDERSAMPLE Experiment 3

models = [MLPClassifier(), XGBClassifier(),  LogisticRegression(max_iter=10000)]

for i in models:
    
    model = i
    over = SMOTE(sampling_strategy=0.05)
    under = RandomUnderSampler(sampling_strategy=0.05)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[adv_stats], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))



F1: 0.5084745762711865 recall: 0.5882352941176471 precision: 0.44776119402985076 predictions: 67.0
F1: 0.5098039215686274 recall: 0.5098039215686274 precision: 0.5098039215686274 predictions: 51.0
F1: 0.4153846153846154 recall: 0.5294117647058824 precision: 0.34177215189873417 predictions: 79.0


/Users/leotsang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [87]:
###Expierment 4

train_2 = train[(train['bpm']>= -0.6) & (train['pts']>= 3) & (train['ht_in']>= 72)] 
models = [MLPClassifier(), XGBClassifier(), LogisticRegression(max_iter=10000)]

for i in models:
    
    model = i
    over = SMOTE(sampling_strategy=0.1)
    under = RandomUnderSampler(sampling_strategy=0.1)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train_2[adv_stats], train_2['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))



F1: 0.47863247863247865 recall: 0.5490196078431373 precision: 0.42424242424242425 predictions: 66.0
F1: 0.48484848484848486 recall: 0.47058823529411764 precision: 0.5 predictions: 48.0
F1: 0.4496124031007752 recall: 0.5686274509803921 precision: 0.3717948717948718 predictions: 78.0


In [88]:
###Expierment 2 Undersmaple by bpm pts, ht_in

train_2 = train[(train['bpm']>= -0.6) & (train['pts']>= 3) & (train['ht_in']>= 72)] 
models = [MLPClassifier(), XGBClassifier(), LogisticRegression(max_iter=10000)]

for i in models:
    
    model = i
    #over = SMOTE(sampling_strategy=0.1)
    #under = RandomUnderSampler(sampling_strategy=0.1)
    #steps = [('o', over), ('u', under)]
    #pipeline = Pipeline(steps=steps)
    #X, y = pipeline.fit_resample(train[adv_stats], train['drafted'])
    model.fit(train_2[adv_stats], train_2['drafted'])
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))




F1: 0.42592592592592593 recall: 0.45098039215686275 precision: 0.40350877192982454 predictions: 57.0
F1: 0.4594594594594595 recall: 0.3333333333333333 precision: 0.7391304347826086 predictions: 23.0
F1: 0.4719101123595506 recall: 0.4117647058823529 precision: 0.5526315789473685 predictions: 38.0


In [92]:
###Expierment 1 Random Under Sample

models = [MLPClassifier(), XGBClassifier(), LogisticRegression(max_iter=10000)]

for i in models:
    
    model = i
    #over = SMOTE(sampling_strategy=0.1)
    under = RandomUnderSampler(sampling_strategy=0.05)
    steps = [('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[adv_stats], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))



F1: 0.39436619718309857 recall: 0.5490196078431373 precision: 0.3076923076923077 predictions: 91.0
F1: 0.4347826086956522 recall: 0.5882352941176471 precision: 0.3448275862068966 predictions: 87.0
F1: 0.40298507462686567 recall: 0.5294117647058824 precision: 0.3253012048192771 predictions: 83.0


In [93]:
###Experiment 3

for i in models:
    
    model = i
    over = SMOTE(sampling_strategy=0.1)
    #under = RandomUnderSampler(sampling_strategy=0.1)
    steps = [('o', over)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[adv_stats], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))



F1: 0.41052631578947363 recall: 0.7647058823529411 precision: 0.2805755395683453 predictions: 139.0
F1: 0.4603174603174603 recall: 0.5686274509803921 precision: 0.38666666666666666 predictions: 75.0
F1: 0.4216216216216216 recall: 0.7647058823529411 precision: 0.291044776119403 predictions: 134.0
